In [1]:
!pip install google-generativeai
!pip install python-dotenv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [19]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import threading
import time
import concurrent

In [20]:
load_dotenv()
api_key_1 = os.getenv('GOOGLE_API_KEY_1')
api_key_2 = os.getenv('GOOGLE_API_KEY_2')
api_key_3 = os.getenv('GOOGLE_API_KEY_3')
api_key_4 = os.getenv('GOOGLE_API_KEY_4')
api_key_5 = os.getenv('GOOGLE_API_KEY_5')
api_key_6 = os.getenv('GOOGLE_API_KEY_6')

api_keys = [
    api_key_1,
    api_key_2,
    api_key_3,
    api_key_4,
    api_key_5,
    api_key_6
]

In [21]:
filenames = [
    "group_1.txt",
    "group_2.txt",
    "group_3.txt",
    "group_4.txt",
    "group_5.txt",
    "group_6.txt",
    "group_7.txt",
    "group_8.txt",
    "group_9.txt",
    "group_10.txt",
    "group_11.txt",
    "group_12.txt",
]

In [22]:
file_contents = []

In [23]:
for filename in filenames:
    file = open(f"./data/{filename}", "r")
    file_content = file.read()
    file.close()
    file_contents.append(file_content)

In [24]:
question = "Kas ma võin autoga sõites alkoholi tarbida?"

In [25]:
system_prompt = "Sa oled seaduste abiline. Kui sulle antud seaduses on küsimusele vastust, siis vasta sellele. Kui vastus puudub, siis vasta '0'"

In [26]:
response_format = """kastuta vastamisel järgmist formaati:
<seaduse nimi>

<sinu vastus>"""

In [27]:
responses = []

In [21]:
group_index = 0
group_count = len(api_keys) * 2

for api_key in api_keys:
    genai.configure(api_key=api_key)

    print(f"progress: {group_index + 1}/{group_count}")
    file_content = file_contents[group_index]
    model = genai.GenerativeModel("gemini-1.5-flash")
    response_text = model.generate_content([
        system_prompt,
        response_format,
        file_content,
        "Küsimus on järgmine:",
        question
    ]).text
    responses.append(response_text)

    group_index += 1

    print(f"progress: {group_index + 1}/{group_count}")
    file_content = file_contents[group_index]
    model = genai.GenerativeModel("gemini-1.5-flash-8b")
    response_text = model.generate_content([
        system_prompt,
        response_format,
        file_content,
        "Küsimus on järgmine:",
        question
    ]).text
    responses.append(response_text)

    group_index += 1

progress: 1/192


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [37]:
group_index = 0
group_count = len(api_keys)

thread_local = threading.local()
responses_lock = threading.Lock()

api_keys_lock = threading.Lock()



def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session

def get_response(data):
    api_keys, file_content, group_index, group_count, system_prompt, response_format, question = data
    
    with api_keys_lock:
        if not api_keys:
            print("No API keys left to process.")
            return
        api_key = api_keys.pop(0)
        
    genai.configure(api_key=api_key)

    
    try:

        # Use generative model
        model = genai.GenerativeModel("gemini-1.5-flash")
        response_text = model.generate_content([
            system_prompt,
            response_format,
            file_content,
            "Küsimus on järgmine:",
            question
        ]).text

        # Safely append to the shared responses list
        with responses_lock:
            responses.append((group_index, response_text))

        print(f"Progress: {group_index + 1}/{group_count}")
    except Exception as e:
        model = genai.GenerativeModel("gemini-1.5-flash-8b")
        response_text = model.generate_content([
            system_prompt,
            response_format,
            file_content,
            "Küsimus on järgmine:",
            question
        ]).text

        # Safely append to the shared responses list
        with responses_lock:
            responses.append((group_index, response_text))

        print(f"Progress: {group_index + 2}/{group_count}")

        
def get_all_responses(api_keys, file_contents, system_prompt, response_format, question):
    group_count = len(api_keys)
    tasks = [
        ( api_keys, file_contents, i, group_count, system_prompt, response_format, question)
        for i in range(group_count)
    ]

    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        executor.map(get_response, tasks)

if __name__ == "__main__":
    
    api_keys = api_keys * 2

    start_time = time.time()
    
    get_all_responses(api_keys, file_contents, system_prompt, response_format, question)
    
    duration = time.time() - start_time

    responses.sort(key=lambda x: x[0])
    for idx, response_text in responses:
        print(f"Response {idx + 1}: {response_text}")

    print(f"I did it in {duration} seconds")


Response 1: Võlaõigusseadus

0

Response 2: <Liiklusseadus>

0

Response 3: Karistusseadustik

Ei. Karistusseadustiku § 424 keelab mootorsõiduki juhtimise joobeseisundis.

Response 4: Meresõiduohutuse seadus

§ 37 lõige 2: Väikelaeva või muu veesõiduki juht ei tohi merel või sisevetel liigeldes olla joobeseisundis.

Response 5: Sotsiaalhoolekande seadus

0

Response 7: Autoveoseadus

0

Response 8: <Kohaliku liikluse seadus>

0

Response 9: Tubakaseadus

0

Response 11: Alkoholi-, tubaka-, kütuse- ja elektriaktsiisi seadus

0

Response 12: Pakendiaktsiisi seadus

0

I did it in 24.187690019607544 seconds


In [38]:
for response in responses:
    print(response)

(0, 'Võlaõigusseadus\n\n0\n')
(1, '<Liiklusseadus>\n\n0\n')
(2, 'Karistusseadustik\n\nEi. Karistusseadustiku § 424 keelab mootorsõiduki juhtimise joobeseisundis.\n')
(3, 'Meresõiduohutuse seadus\n\n§ 37 lõige 2: Väikelaeva või muu veesõiduki juht ei tohi merel või sisevetel liigeldes olla joobeseisundis.\n')
(4, 'Sotsiaalhoolekande seadus\n\n0\n')
(6, 'Autoveoseadus\n\n0\n')
(7, '<Kohaliku liikluse seadus>\n\n0\n')
(8, 'Tubakaseadus\n\n0\n')
(10, 'Alkoholi-, tubaka-, kütuse- ja elektriaktsiisi seadus\n\n0\n')
(11, 'Pakendiaktsiisi seadus\n\n0\n')


In [40]:
print(api_keys)

[]


In [39]:
genai.configure(api_key=api_keys[0])
model = genai.GenerativeModel("gemini-1.5-pro")

prompt = (
    ["Sulle antakse mitme erineva mudeli vastused erinevatest seadustest, tsiteeri mulle mudeli vastuseid, mis pole '0'"] +
    [response_format] +
    responses + 
    ["Küsimus on järgnev:"] +
    [question]
)

best_answer = model.generate_content(prompt)

IndexError: list index out of range

In [15]:
print(best_answer.text)

NameError: name 'best_answer' is not defined

In [20]:
%%html
<html>
<head>
  <link rel="stylesheet" href="styles.css">
</head>
<body>
<div class="header">
    <h1>juur<span>.ai</span></h1>
</div>

<div class="input-footer">
    <textarea type="text" placeholder="Ask anything" class="input-text glassmorphism"> </textarea>
</div>
</body>
</html>